In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH, DEST_PATH = utils.main_processor(
    data_dir='..\data',  # search project ..\data
    downloads_dir='',  # None searchs Downloads dir, '' omits search
    downloads_limit=0,  # search the first 10 files
    clean_name_override='df_perf_ratios.pkl',  # override filename
    start_file_pattern='df_OHLCV_', # search for files starting with 'df_'
)



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\python310.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src']


**Available 'df_OHLCV_' files:**

- (1) `[DATA]` `df_OHLCV_2025-03-07_clean.pkl` <span style='color:#00ffff'>(45.60 MB, 2025-03-09 12:53)</span>

- (2) `[DATA]` `df_OHLCV_2025-03-10_clean.pkl` <span style='color:#00ffff'>(45.55 MB, 2025-03-10 20:09)</span>


Input a number to select file (1-2)



    **Selected paths:**
    - Source: `..\data\df_OHLCV_2025-03-10_clean.pkl`  
    - Destination: `..\data\df_perf_ratios.pkl`
    

In [3]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle(SOURCE_PATH)

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
AAPL   2025-03-10     227.48
       2025-03-07     239.07
       2025-03-06     235.33
       2025-03-05     235.74
       2025-03-04     235.93
...                      ...
FCOM   2024-03-15      47.62
       2024-03-14      48.10
       2024-03-13      48.22
       2024-03-12      48.20
       2024-03-11      47.87

[620000 rows x 1 columns]

In [4]:
import utils

list_dfs = utils.get_latest_dfs(df, DAYS_RATIO)

In [5]:
all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  \
A      -5.505585    -5.855561   0.335729   -6.734780    -7.002462   0.300426   
AA     -5.666268    -6.069974   0.374360   -2.033830    -2.547147   0.701730   
AAL    -9.758861    -9.581713   0.204881   -7.367966    -7.556580   0.281696   
AAON   -5.291580    -5.546311   0.302506   -3.217010    -3.543728   0.480896   
AAP    -3.486874    -3.946222   0.472947   -3.332664    -3.740184   0.507988   
...          ...          ...        ...         ...          ...        ...   
ZM     -6.501352    -6.330959   0.227003   -2.215518    -2.891289   0.619095   
ZS     -3.757921    -4.394813   0.570578    0.475289     0.684074   1.080152   
ZTO     0.644115     0.947906   1.109089    1.342331     1.980133   1.228681   
ZTS     7.192651    14.298137   3.187329   -0.201828    -0.256705   0.965810   
ZWS    -2.787690    -3.419556   0.635059   -4.457622    -4.902777   0.482494   

In [6]:
# Save the combined DataFrame to pickle file
combined_df.to_pickle(DEST_PATH)
print(f"DataFrame successfully saved to {DEST_PATH}")

DataFrame successfully saved to ..\data\df_perf_ratios.pkl


In [7]:
_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']

from tabulate import tabulate

# Create a DataFrame with only the selected symbols
selected_df = combined_df.loc[_symbols]

# Display the formatted table
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+==============+===============+=============+==============+===============+=============+==============+===============+=============+===============+================+==============+===============+================+==============+
| AAPL |      -4.7174 |       -5.0886 |      0.3901 |      -0.2793 |       -0.3698 |      0.9520 |      -1.3574 |       -1.7114 |      0.7926 |        0.4225 |         0.5726 |       1.0754 |        1.1192 |         1.6548 |       1.2161 |
+------+--------------+---------------+-